In [1]:
import cv2
import numpy as np

def generate_aruco_marker(dictionary_id, marker_id, marker_size, output_filename):
    """
    Generate an ArUco marker and save it as an image file.

    Args:
        dictionary_id (int): ArUco dictionary ID (e.g., cv2.aruco.DICT_6X6_250).
        marker_id (int): ID of the marker to generate.
        marker_size (int): Size of the marker in pixels (e.g., 200).
        output_filename (str): File name to save the marker image.
    """
    # Get the specified ArUco dictionary
    aruco_dict = cv2.aruco.getPredefinedDictionary(dictionary_id)
    print("uo",aruco_dict)


    # Create a blank marker image
    marker_image = np.zeros((marker_size, marker_size), dtype=np.uint8)

    # Generate the marker
    marker_image = cv2.aruco.generateImageMarker(
            aruco_dict, marker_id, marker_size, marker_image, 1
        )

    # Save the marker image
    cv2.imwrite(output_filename, marker_image)
    print(f"ArUco marker saved as {output_filename}")


def calculate_distance(marker_corners, marker_real_width, focal_length):
    """
    Calculate the distance to the ArUco marker.
    Args:
        marker_corners (numpy.ndarray): Detected marker corners.
        marker_real_width (float): Real-world width of the marker (e.g., in cm or inches).
        focal_length (float): Focal length of the camera.
    Returns:
        float: Distance to the marker.
    """
    # Extract the top-left and top-right points to calculate the perceived width
    top_left = marker_corners[0][0]
    top_right = marker_corners[0][1]
    perceived_width = np.linalg.norm(top_right - top_left)
    
    # Distance calculation formula
    distance = (marker_real_width * focal_length) / perceived_width
    return distance


def main():
    # Generate an ArUco marker
    dictionary_id = cv2.aruco.DICT_6X6_250  # Choose an ArUco dictionary
    marker_id = 42  # ID of the marker
    marker_size = 300  # Marker size in pixels
    output_filename = "aruco_marker_42.png"  # Output file name
    #generate_aruco_marker(dictionary_id, marker_id, marker_size, output_filename)

    # Distance measurement setup
    aruco_dict = cv2.aruco.getPredefinedDictionary(dictionary_id)
    parameters = cv2.aruco.DetectorParameters()

    # Camera calibration parameters
    focal_length = 679.1592725617968  
    marker_real_width = 4.0  

    # Start video capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect ArUco markers
        corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

        if ids is not None:
            # Draw detected markers

            cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            for marker_corners in corners:
                # Calculate distance
                distance = calculate_distance(marker_corners[0], marker_real_width, focal_length)

                # Display distance on the frame
                top_left = tuple(marker_corners[0][0].astype(int))
                cv2.putText(frame, f"Distance: {distance:.2f} cm", top_left, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('ArUco Distance Measurement', frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
